## Jrfapp tutorial 2 
## Inverting a synthetic Model using a real dataset with noise:

First we need to import Initialize and Jrfapp_station. You can perform it by:

In [2]:
from jrfapp.main_classes import Initialize
from jrfapp.main_classes import Jrfapp_station
import os

In this run we want to read a real dataset from a station and perform a synthetic with 50.0 % noise level. 
The first step is to initialize the code. However, in this run we need to define the folder of our netwwork. This folder 
can contain several station data. Each station data is in a folder with a name according to an input file which define 
the station name and coordinates of all station of corresponding network. In this example i used the dataset of a permanent
station in the MAKRAN network. We need to give the coordinates file name of this network in the Initialize class. The Initialize
class is then defined as below.

In [ ]:
## assigning folders and files needed for run.
# first define the package folder you downloaded from git.
package_folder = '/home/soroush/rf_shallow_codes/my_py_rf/Jrfapp_proj'
station_coordinate_file_path = os.path.join(package_folder, 'makran_coordinates')
data_folder_path = os.path.join(package_folder, 'makran_data')
output_folder_path = os.path.join(package_folder, 'jrfapp_output_syn_wnoise')
model_folder_path = os.path.join(package_folder, 'model_folder')

In [ ]:
init_obj = Initialize(network_name='MAKRAN', 
                               station_coordinate_file= station_coordinate_file_path,
                               ,data_folder=data_folder_path, 
                               layering= [3, 4, 3], 
                  output_folder=output_folder_path, 
                   model_name= 'halfspace',
                   random_seed= 250)

The makran_coordinates file contains the station name, latitude and longitude. 
for example 
> CHBR    25.595      60.482

is one of the lines in makran_coordinates that correspond to CHBR station.

The data_folder is where we saved the dataset of CHBR station. the data_folder structures must be as follow:
> {data_folder}/{station_folder}/{event_folder}/*BHZ.SAC *BHE.SAC *BHN.SAC
> data_folder/CHBR/2017119105924/2017119105924_BHE.SAC,
> data_folder/CHBR/2017119105924/2017119105924_BHN.SAC,  
> data_folder/CHBR/2017119105924/2017119105924_BHZ.SAC.



Now we need to define our synthetic model and Jrfapp_stobj for this station as follows:

In [ ]:
which_to_pert = []
which_to_pert.append([6, 12,  -0.6])
which_to_pert.append([18, 24,  0.6])
which_to_pert.append([30, 36, -0.6])
which_to_pert.append([50, 60,  0.6])
init_obj.create_synthetic(which_to_pert)
jrfapp_stobj = cc.Jrfapp_station(init_obj, name = 'CHBR', 
                                      noise_level= 50.0)

Next we can invert the data by either grid_search or PSO method. But first lets save the Jrfapp_stobj as the processes
for calculating RF and Apparent velocity and stacking them can take long time and we dont want to repeat this processes 
every time. 

In [ ]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='syn_with_noise_gs_bf_inv')

In [ ]:
we can later load this data by pickle.

In [ ]:
jrfapp_stobj_file_name = '/home/soroush/rf_shallow_codes/my_py_rf/pkg_test_full_syn_chbr_gs/syn_with_noise_gs_bf_inv'
with open(jrfapp_stobj_file_name, 'rb') as f1:
    jrfapp_stobj = pickle.load(f1)

Now we can invert synthetic data by:

In [ ]:
jrfapp_stobj.invert_data(inv_method = 'grid_search',
                     stack_name = 'synthetic',
                     ndivide_list = [-1, 1, -2, 2, -3, 3, -4, 4], 
                     nmodel = 6, nthread = 6, 
                     finer_ndivide= [-2, 2, -3, 3, -4, 4, -5, 5])

In [ ]:
jrfapp_stobj_file_name = jrfapp_stobj.save_file(file_name='syn_with_noise_gs_af_inv')

Inversion using PSO:

In [ ]:
jrfapp_stobj.invert_data(inv_method = 'PSO',
                stack_name = 'synthetic',
                ndivide_list = [-1, 1, -2, 2, -3, 3, -4, 4], 
                PSO_nparticle = 4, PSO_maxiter = 2, 
                nthread = 6)

<div class="alert alert-block alert-info">
<b>Tip:</b> Again for simplicity I run the inversion with 6 models for grid_search and 4 particles in 2 iterations for the PSO run.
</div>